# <span style="color:rgb(255, 0, 255)"> Lab handling data imbalance 
<span style="color:rgb(255, 0, 255)"> Ainara Guerra
    

**But first, libraries:**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.ticker as mk
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression

import os #we will use the function listdir to list files in a folder
import math #to apply absolute value

### <span style="color:rgb(255, 0, 255)">--- Load the database</span>

In [2]:
data = pd.read_csv('Customer-Churn.csv')

### <span style="color:rgb(255, 0, 255)">--- Let's look at its main features (head, shape, info).

In [3]:
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [4]:
data.shape
# It has 7043 rows x 16 columns, not so bad

(7043, 16)

In [5]:
data.info()
# Here we can see it apparently doesn't have nulls, but we should deep into each column to see how their values are distributed.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [6]:
#let's see first if we have any duplicated rows with nan values in those rows aka no info in that entry
duplicate_rows = data[data.duplicated()]
duplicate_rows

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
100,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.20,20.2,No
772,Female,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,19.90,19.9,Yes
885,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.05,20.05,No
964,Male,0,No,No,1,Yes,No,No,No,No,No,No,Month-to-month,45.70,45.7,Yes
987,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,19.75,19.75,No
1338,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.15,20.15,Yes
1491,Female,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,19.55,19.55,No
1739,Male,0,No,No,1,Yes,No,No,No,No,No,No,Month-to-month,69.90,69.9,Yes
1932,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.20,20.2,No
2121,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.45,20.45,No


In [7]:
data.isna().sum()
# another way of see the nulls

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### <span style="color:rgb(255, 0, 255)">--- Are the columns names on point?

In [8]:
data.columns
# Seem pretty good to me

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

***

### <span style="color:rgb(255, 0, 255)">--- Lab instructions starts here

#### <span style="color:rgb(255, 0, 255)"> ---- **Read that data into Python and call the dataframe churnData:**

In [9]:
ChurnData = data.copy()

#### <span style="color:rgb(255, 0, 255)"> ----  Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function:

In [10]:
#Checking dtypes of the dataset:
ChurnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [11]:
ChurnData["TotalCharges"]
# It is indeed a number.

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [12]:
# Let's explore it before turning into a numeric
ChurnData["TotalCharges"].value_counts()


          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

In [13]:
# We see there are 11 values with empty spaces. Looks good to me to turn it into zero in we don't have value for that. 
#It could also be a good idea to drop them since we don't have much
ChurnData["TotalCharges"].replace(" ", "0", inplace=True)

In [14]:
ChurnData["TotalCharges"].value_counts()
# It worked

0         11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

In [15]:
# Now er can convert it to numeric:
ChurnData["TotalCharges"] = pd.to_numeric(ChurnData["TotalCharges"], errors="coerce")

In [16]:
# let's see if it worked by checking the NaNs:
ChurnData["TotalCharges"].isna().sum()
#It worked

0

#### <span style="color:rgb(255, 0, 255)"> ---- Check for null values in the dataframe. Replace the null values.

In [17]:
# We saw earlier that there weren't NaNs per se. 
# Let's check value counts of each column to see if theres anything we need to tackle (detective mode on🔍)

# 1. let's remember the colums:
ChurnData.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [18]:

for col in ChurnData.columns:
    print(ChurnData[col].value_counts())

Male      3555
Female    3488
Name: gender, dtype: int64
0    5901
1    1142
Name: SeniorCitizen, dtype: int64
No     3641
Yes    3402
Name: Partner, dtype: int64
No     4933
Yes    2110
Name: Dependents, dtype: int64
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64
Yes    6361
No      682
Name: PhoneService, dtype: int64
No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64
No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64
No                     3095
Yes                    2422
No internet service    1526
Name: DeviceProtection, dtype: int64
No                     3473
Yes                    2044
No internet service    1526
Name: TechSupport, dtype: int64
No                     2810
Yes                    2707
No internet service    1526
Name: StreamingTV, 

In [19]:
# This looks good to me, but what if we have some calues = " ", as we had in TotalCharges that weren't spotted?
# Let's discover if there is anything else:
empty_columns = (ChurnData == " ").sum()

# Iterate over the columns and print the count of empty values per column
for column, count in empty_columns.iteritems():
    if count > 0:
        print(f"Column '{column}' has {count} empty value(s).")
    if empty_columns.sum() == 0:
        print(f"Column '{column}' has not empty values.")
        
#Apparently, looks clean to me. 

Column 'gender' has not empty values.
Column 'SeniorCitizen' has not empty values.
Column 'Partner' has not empty values.
Column 'Dependents' has not empty values.
Column 'tenure' has not empty values.
Column 'PhoneService' has not empty values.
Column 'OnlineSecurity' has not empty values.
Column 'OnlineBackup' has not empty values.
Column 'DeviceProtection' has not empty values.
Column 'TechSupport' has not empty values.
Column 'StreamingTV' has not empty values.
Column 'StreamingMovies' has not empty values.
Column 'Contract' has not empty values.
Column 'MonthlyCharges' has not empty values.
Column 'TotalCharges' has not empty values.
Column 'Churn' has not empty values.


#### <span style="color:rgb(255, 0, 255)"> ---- Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
    
1. Scale the features either by using normalizer or a standard scaler.
2. Split the data into a training set and a test set.
3. Fit a logistic regression model on the training data.
4. Check the accuracy on the test data.

In [20]:
data_df = ChurnData[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
data_df

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65
...,...,...,...,...
7038,24,0,84.80,1990.50
7039,72,0,103.20,7362.90
7040,11,0,29.60,346.45
7041,4,1,74.40,306.60


In [21]:
# 1. Scale the features either by using normalizer or a standard scaler.
transformer = MinMaxScaler().fit(data_df)
data_df_minmax = transformer.transform(data_df)
data_df_norm = pd.DataFrame(data_df_minmax,columns=data_df.columns)
data_df_normalized = data_df_norm.copy()
data_df_normalized.shape

(7043, 4)

In [22]:
# 2. Split the data into a training set and a test set.
X = data_df_normalized
y = ChurnData["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 

In [23]:
# Create an instance of Logistic Regression
logreg = LogisticRegression()

# Train the model using the training data
logreg.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = logreg.score(X_test, y_test)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.7989778534923339


In [24]:
from sklearn.metrics import classification_report
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df

,precision,recall,f1-score,support
No,0.819120,0.929017,0.870614,1282.000000
Yes,0.703583,0.450939,0.549618,479.000000
accuracy,0.798978,0.798978,0.798978,0.798978
macro avg,0.761351,0.689978,0.710116,1761.000000
weighted avg,0.787693,0.798978,0.783302,1761.000000


<!-- <span style="color:rgb(255, 0, 255)"> --- **Read that data into Python and call the dataframe churnData:** -->